In [1]:
%run "../HyperParameterOpt/GenerateExperiments/res_experiment.py"
from scipy import sparse
import networkx as nx
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D 
import os
import graph_tool.all as gt
import matplotlib.pyplot as plt
import numpy as np
import gc

plt.switch_backend("cairo")


In [2]:
plt.close("all")
gc.collect()

4

# Notation Explanation Plots

A notebook of plots to help explain $\mathbf{u}(t)$, $\mathbf{\hat{u}}(t)$, $\mathbf{r}(t)$, the processing network and the aggregate network.

### Parameters

In [3]:
DIFF_EQ_PARAMS = {
    "x0": [-20, 10, -.5],
    "begin": 0,
    "end": 85,
    "timesteps": 85000,
    "train_per": .889,
    "solver": lorenz_equ,
    "clip": 40
}
RES_PARAMS = {
    "uniform_weights": True,
    "solver": "ridge",
    "ridge_alpha":.0001,
    "signal_dim": 3,
    "network": "random graph",
    "res_sz": 50,
    "activ_f": np.tanh,
    "connect_p": .1,
    "spect_rad": 50,
    "gamma": 5,
    "sigma": 0.05,
    "sparse_res": True,
}

### Functions

In [4]:
def erdos(mean_degree,n=None):
    """ Erdos-Renyi random graph.
    Parameter
        mean_degree     (int): specific to this topology
        n               (int): n is the size of the network
    """
    if n is None:
        n = np.random.randint(smallest_network_size,biggest_network_size)
    p = mean_degree/n
    A = nx.adj_matrix(nx.erdos_renyi_graph(n,p)).T
    return sparse.dok_matrix(A)

def remove_edges(A, p):
    """ Randomly removes 'p' percent of edges from a sparse matrix 'A'
    """
    nedges = floor(p*np.sum(A != 0))
    A = A.todok()
    # Remove Edges
    keys = list(A.keys())
    remove_idx = np.random.choice(range(len(keys)),size=nedges, replace=False)
    remove = [keys[i] for i in remove_idx]
    for e in remove:
        A[e] = 0
    return A

def nx_to_gt(G):
    nx.write_graphml(G, "temp.xml")
    g = gt.load_graph("temp.xml")
    os.remove("temp.xml")
    return g

def simple_gt_plot(G, mplaxis, colors=None, edge_colors=None, vertex_size=1.0):
    g = nx_to_gt(G)
    if colors is not None:
        color_map = {i:colors[i] for i in range(G.number_of_nodes())}
        # Create new vertex property
        plot_color = g.new_vertex_property('vector<double>')
        # add that property to graph
        g.vertex_properties['plot_color'] = plot_color
        # assign a value to that property for each node of that graph
        for v in g.vertices():
            plot_color[v] = color_map[v]
        vertex_fill_color = g.vertex_properties['plot_color']
    else:
        vertex_fill_color=None
        
    if edge_colors is not None:
        edge_color_map = {e:edge_colors[i] for i,e in enumerate(g.edges())}
        # Create new edge property
        plot_color = g.new_edge_property('vector<double>')
        # add that property to graph
        g.edge_properties['plot_color'] = plot_color
        # assign a value to that property for each node of that graph
        for v in g.edges():
            plot_color[v] = edge_color_map[v]
        edge_color = g.edge_properties['plot_color']
    else:
        edge_color=None
    
    if mplaxis is None:
        fig, mplaxis = plt.subplots(1, 1, figsize=(12, 11.5))

    gt.graph_draw(
        g, 
        vertex_fill_color=vertex_fill_color, 
        vertex_size=vertex_size, 
        edge_color=edge_color,
        mplfig=mplaxis
    )
    mplaxis.set_axis_off()


In [5]:

def notation_expl(rc, R, U, Uhat, test_t):
    acc_dur = how_long_accurate(U, Uhat, tol=TOL)
    # Node colors
    nc = np.random.rand(rc.res_sz, 3)
    titlesz = 14
    linewidth = 0.5
    fig = plt.figure(figsize=(17.5,5))
    gridspec = GridSpec(2,4)
    
    subplotspec = gridspec.new_subplotspec((0, 0), rowspan=1, colspan=1)
    ax = fig.add_subplot(subplotspec)
    
    ax.set_title("Processing Network", fontsize=titlesz)
    simple_gt_plot(nx.Graph(rc.res), ax, colors=nc)

    subplotspec = gridspec.new_subplotspec((0, 1), rowspan=1, colspan=2)
    ax = fig.add_subplot(subplotspec)
    ax.set_title("Processing Network Response $\\mathbf{r}(t)$", fontsize=titlesz)
    for i in range(n):
        ax.plot(train_t, R[:, i], lw=linewidth, c=nc[i])
    ax.set_yticks([-1, 1])
    ax.set_ylabel("Reservoir Node States")
    tks = np.arange(40, 90, 10)
    ax.set_xticks(tks)
    ax.set_xticklabels(tks - 40)

    subplotspec = gridspec.new_subplotspec((1, 0), rowspan=1, colspan=1)
    ax = fig.add_subplot(subplotspec)
    ax.set_title("Aggregate Network", fontsize=titlesz)
    aggA = np.array(rc.res + rc.sigma*rc.W_in @ rc.W_out)
    # Remove self edges
    for i in range(aggA.shape[0]):
        aggA[i,i]=0
        
    G = nx.Graph(aggA)
    labels = nx.get_edge_attributes(G,'weight')
    eweights = np.log([np.abs(labels[e]) + 1.0 for e in G.edges()])
    eweights /= np.max(eweights)
    #eweights = 1 - eweights
    ecolors = np.vstack((eweights, eweights, eweights)).T
    
    teal = np.array([0,128,128])/256
    node_colors = [teal for i in range(G.number_of_nodes())]
    simple_gt_plot(G, ax, colors=node_colors, edge_colors=ecolors, vertex_size=0.17)

    subplotspec = gridspec.new_subplotspec((1, 1), rowspan=1, colspan=2)
    ax = fig.add_subplot(subplotspec)
    ax.set_title("Aggregate Network Prediction $\\mathbf{\\hat{u}}(t)$", fontsize=titlesz)
    ymax = np.max([np.max(Uhat), np.max(U)])
    ymin = np.min([np.min(Uhat), np.min(U)])
    ax.plot(test_t, U.T, "gray", lw=linewidth,)
    ax.plot(test_t[0], U.T[0,0], "gray", label="True: $\\mathbf{u}(t)$")
    ax.plot(test_t, Uhat.T, "teal", lw=linewidth,)
    ax.plot(test_t[0], Uhat.T[0,0], "teal", label="Predicted: $\\mathbf{\\hat{u}}(t)$")
    ax.plot(np.ones(10)*test_t[acc_dur], np.linspace(ymin, ymax, 10), "--", c="r", lw=linewidth)
    ax.set_xlabel("Time (a.u.)")
    ax.set_ylabel("Location (a.u.)")
    ts_tks = np.arange(80, 86, 1)
    ax.set_yticks([np.round(ymin), np.round(ymax)])
    ax.set_xticks(ts_tks)
    ax.set_xticklabels(ts_tks - 40)
    locs, labels = ax.get_xticks(), ax.get_xticklabels()
    ax.set_xticks(list(locs) + [test_t[acc_dur]])
    ax.set_xticklabels( list(labels) + ["VPT"])
    ax.legend(loc=(0.8, .65))
    
    subplotspec = gridspec.new_subplotspec((0, 3), rowspan=2, colspan=1)
    ax = fig.add_subplot(subplotspec, projection='3d')
    
    x,y,z = U[0,:], U[1,:], U[2, :]
    xh,yh,zh = Uhat[0,:], Uhat[1,:], Uhat[2, :]
    ax.plot3D(x, y, z, c="gray",alpha=.8)
    ax.plot3D(xh, yh, zh, c="teal",alpha=.8)
    ax.axis("off")
    ax.set_title("True v.s. Predicted Orbit in 3D", fontsize=titlesz)
    ax.set_xlim(-15,19)
    ax.set_ylim(-15, 19)
    ax.set_zlim(20, 41)

    plt.tight_layout()
    return fig

In [6]:
DIFF_EQ_PARAMS["x0"] = random_lorenz_x0()
train_t, test_t, u = rc_solve_ode(DIFF_EQ_PARAMS)

In [7]:

rho = 0
mean_degree = 0
remove_p = 0.0

n = RES_PARAMS["res_sz"]
RES_PARAMS["spect_rad"] = rho
A = remove_edges(erdos(mean_degree, n), remove_p)
A = sparse.lil_matrix(A)

In [12]:
nx.is_connected(nx.Graph(A))


False

In [20]:
rc = ResComp(A, **RES_PARAMS)
rc.state_0 = 2*np.random.rand(rc.res_sz) - 1
r0 = rc.state_0
err, R = rc.fit(train_t, u, return_states=True)
U = u(test_t)
Uhat = rc.predict(test_t)
acc_dur = how_long_accurate(U, Uhat, tol=TOL)
print(f"VPT: {acc_dur}")

/Users/djpassey/Code/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


VPT: 2648


In [21]:
f = notation_expl(rc, R, U, Uhat, test_t)
plt.savefig("zero_edges.png", dpi=300)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "dpi" which is no longer supported as of 3.3 and will become an error two minor releases later


In [30]:
rho = 100
mean_degree = 0.2
remove_p = 0.9


n = RES_PARAMS["res_sz"]
RES_PARAMS["spect_rad"] = rho
A = erdos(mean_degree, n)
A = sparse.lil_matrix(A)

print(f"Contains {np.sum(A != 0)} edges")

Contains 10 edges


In [31]:


rc_sp = ResComp(A, **RES_PARAMS)
rc_sp.state_0 = 2*np.random.rand(rc_sp.res_sz) - 1
r0 = rc_sp.state_0
err, Rsp = rc_sp.fit(train_t, u, return_states=True)
Usp = u(test_t)
Uhat_sp = rc_sp.predict(test_t)
acc_dur_sp = how_long_accurate(Usp, Uhat_sp, tol=TOL)
print(f"VPT: {acc_dur_sp}")

VPT: 3491


In [32]:
f = notation_expl(rc_sp, Rsp, Usp, Uhat_sp, test_t)
plt.savefig("spect_rad_eq_100.png", dpi=300)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "dpi" which is no longer supported as of 3.3 and will become an error two minor releases later


In [22]:
nc = np.random.rand(n, 3)
nx.draw_spring(nx.DiGraph(rc_sp.res), node_color=nc, node_size=60, arrows=False)

In [24]:
aggA = np.array(rc_sp.res + rc_sp.sigma*rc_sp.W_in @ rc_sp.W_out)
G = nx.DiGraph(aggA)
labels = nx.get_edge_attributes(G,'weight')
eweights = [np.abs(labels[e]) for e in G.edges()]
eweights /= np.max(eweights)
eweights = 1 - eweights
ecolors = np.vstack((eweights, eweights, eweights)).T
nx.draw_spring(G, edge_color=ecolors, node_color=nc, node_size=60, arrows=False)

In [25]:
def knngraph(A, k):
    n, _ = A.shape
    G = nx.DiGraph()
    for i, ai in enumerate(A):
        nearestnodes = np.argsort(ai)[::-1]
        for j in range(k):
            neighbor = nearestnodes[j]
            G.add_edge(neighbor, i, weight=A[i, neighbor])
    return G       

In [26]:
np.argsort(aggA[0])

array([45,  5, 12,  7, 11, 21, 26, 33, 46,  3, 32, 27, 25, 23, 37,  2, 35,
       38, 15, 31,  0, 47,  9, 19, 36, 22, 42, 49,  6, 40, 39, 44, 18,  8,
       14, 10, 28, 17, 20,  4, 29, 13, 34, 48, 43, 24,  1, 16, 41, 30])

In [27]:
knnG = knngraph(aggA, 5)
labels = nx.get_edge_attributes(knnG,'weight')
eweights = [np.abs(labels[e]) for e in knnG.edges()]
eweights /= np.max(eweights)
eweights = 1 - eweights
ecolors = np.vstack((eweights, eweights, eweights)).T
nx.draw_spring(knnG, edge_color=ecolors, node_color="k", node_size=60, arrows=False)